In [20]:
n = int(input("Enter N:"))
# n= 3
environment = {}
k = 1
for i in range(n):
    for j in range(n):
        environment[(i,j)] = f"s{k}"
        k+=1

print(environment)



{(0, 0): 's1', (0, 1): 's2', (0, 2): 's3', (0, 3): 's4', (0, 4): 's5', (1, 0): 's6', (1, 1): 's7', (1, 2): 's8', (1, 3): 's9', (1, 4): 's10', (2, 0): 's11', (2, 1): 's12', (2, 2): 's13', (2, 3): 's14', (2, 4): 's15', (3, 0): 's16', (3, 1): 's17', (3, 2): 's18', (3, 3): 's19', (3, 4): 's20', (4, 0): 's21', (4, 1): 's22', (4, 2): 's23', (4, 3): 's24', (4, 4): 's25'}


In [21]:
def return_index(s : str):
    for k in environment:
        if environment[k] == s:
            return k

In [ ]:
goal = input("Enter Goal state:")
start = "s1"
# start = "s7"
# goal = "s3"
if (start == goal):
    start = "s2"

def action_reward(action : int, i_p : int , j_p : int):#returns reward and next state index
    if action==0:
        i_n = i_p-1 #i_new
        if i_n<0:
            return [-100,i_p,j_p]
        elif environment[(i_n,j_p)] == goal:
            return [100,i_n,j_p]
        else:
            return [0,i_n,j_p]
    elif action == 1:
        j_n = j_p+1
        if j_n>=n:
            return [-100,i_p,j_p]
        elif environment[(i_p,j_n)] == goal:
            return [100,i_p,j_n]
        else:
            return [0,i_p,j_n]
    elif action == 2:
        i_n = i_p+1
        if i_n>=n:
            return [-100,i_p,j_p]
        elif environment[(i_n,j_p)] == goal:
            return [100,i_n,j_p]
        else:
            return [0,i_n,j_p]
    elif action==3:
        j_n = j_p-1
        if j_n<0:
            return [-100,i_p,j_p]
        elif environment[(i_p,j_n)] == goal:
            return [100,i_p,j_n]
        else:
            return [0,i_p,j_n]
    
rewards = {}
k = 1
i = 0
j = 0 
for i in range(n):
    for j in range(n):
        rewards[f"s{k}"] = [
            action_reward(0,i,j)[0],#north
            action_reward(1,i,j)[0],#east
            action_reward(2,i,j)[0],#south
            action_reward(3,i,j)[0]#west
        ]
        k=k+1

import copy
Q_table = copy.deepcopy(rewards)
print(rewards)

{'s1': [-100, 100, 0, -100], 's2': [-100, 0, 0, 0], 's3': [-100, 0, 0, 100], 's4': [-100, 0, 0, 0], 's5': [-100, -100, 0, 0], 's6': [0, 0, 0, -100], 's7': [100, 0, 0, 0], 's8': [0, 0, 0, 0], 's9': [0, 0, 0, 0], 's10': [0, -100, 0, 0], 's11': [0, 0, 0, -100], 's12': [0, 0, 0, 0], 's13': [0, 0, 0, 0], 's14': [0, 0, 0, 0], 's15': [0, -100, 0, 0], 's16': [0, 0, 0, -100], 's17': [0, 0, 0, 0], 's18': [0, 0, 0, 0], 's19': [0, 0, 0, 0], 's20': [0, -100, 0, 0], 's21': [0, 0, -100, -100], 's22': [0, 0, -100, 0], 's23': [0, 0, -100, 0], 's24': [0, 0, -100, 0], 's25': [0, -100, -100, 0]}


In [23]:
import random
alpha = 1.0
gamma = 0.9

i = 0
for i in range(20):
    state = start
    while state != goal:
        action = random.randint(0,3)
        rtplus1 = rewards[state][action]
        old_Q = Q_table[state][action]

        (a,b) = return_index(state)
        after_action = action_reward(action,a,b)
        if after_action[0] == -100:
            next_maxQ = 0
            Q_table[state][action] = rtplus1
            continue
        next_state = environment[(after_action[1],after_action[2])]
        # print(f"{state}-->{next_state}")
        next_maxQ = max(Q_table[next_state])
        # print(next_maxQ)
        new_Q = ((1.0-alpha)*old_Q )+ (alpha*(rtplus1+(gamma*next_maxQ)))
        Q_table[state][action] = new_Q
        state = next_state
        # print(rewards)
    print(Q_table)


        

{'s1': [-100, 100.0, 0, -100], 's2': [-100, 0, 0, 0], 's3': [-100, 0, 0, 100], 's4': [-100, 0, 0, 0], 's5': [-100, -100, 0, 0], 's6': [0, 0, 0, -100], 's7': [100, 0, 0, 0], 's8': [0, 0, 0, 0], 's9': [0, 0, 0, 0], 's10': [0, -100, 0, 0], 's11': [0, 0, 0, -100], 's12': [0, 0, 0, 0], 's13': [0, 0, 0, 0], 's14': [0, 0, 0, 0], 's15': [0, -100, 0, 0], 's16': [0, 0, 0, -100], 's17': [0, 0, 0, 0], 's18': [0, 0, 0, 0], 's19': [0, 0, 0, 0], 's20': [0, -100, 0, 0], 's21': [0, 0, -100, -100], 's22': [0, 0, -100, 0], 's23': [0, 0, -100, 0], 's24': [0, 0, -100, 0], 's25': [0, -100, -100, 0]}
{'s1': [-100, 100.0, 0.0, -100], 's2': [-100, 0, 0, 0], 's3': [-100, 0, 0, 100], 's4': [-100, 0, 0, 0], 's5': [-100, -100, 0, 0], 's6': [0, 90.0, 0, -100], 's7': [100.0, 0, 0.0, 0], 's8': [0, 0, 0, 0], 's9': [0, 0, 0, 0], 's10': [0, -100, 0, 0], 's11': [0, 0, 0, -100], 's12': [90.0, 0, 0, 0], 's13': [0, 0, 0, 0], 's14': [0, 0, 0, 0], 's15': [0, -100, 0, 0], 's16': [0, 0, 0, -100], 's17': [0, 0, 0, 0], 's18': [0,

In [24]:
def create_policy():
    policy = {}
    # print(Q_table)
    for state in Q_table:
        policy[state] = Q_table[state].index(max(Q_table[state]))
    print(policy)
    return policy
    
def construct_path(start,goal):
    path = ""
    policy = create_policy()
    state = start 
    path = path+f"{state}"
    while state!=goal:
        action = policy[state]
        a,b = return_index(state)
        next_state_index = action_reward(action,a,b)
        next_state = environment[(next_state_index[1],next_state_index[2])]
        state = next_state
        path = path+f"-->{state}"
    return(path)

start = input("Enter start state:")
path = construct_path(start,goal)

print(path)

    

{'s1': 1, 's2': 1, 's3': 3, 's4': 3, 's5': 3, 's6': 0, 's7': 0, 's8': 0, 's9': 0, 's10': 0, 's11': 0, 's12': 0, 's13': 0, 's14': 0, 's15': 0, 's16': 0, 's17': 0, 's18': 0, 's19': 0, 's20': 0, 's21': 0, 's22': 0, 's23': 0, 's24': 0, 's25': 0}
s25-->s20-->s15-->s10-->s5-->s4-->s3-->s2


In [ ]:
# environment = [
#     ["s1","s2","s3"],
#     ["s4","s5","s6"],
#     ["s7","s8","s9"]
# ]

# def get_index(s: str):
#     for i in range(len(environment)):
#         for j in range(len(environment)):
#             if environment[i][j]==s:
#                 return (i,j)

#stste : [north,east,south,west]
# rewards = {
#     "s1":[0,0,0,0],
#     "s2":[0,100,0,0],
#     "s3":[0,0,0,0],
# }
# "S7".lower()

's7'